In [53]:
import os
import warnings
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import seaborn as sns
from dotenv import load_dotenv

In [54]:
# # Warning 무시
# warnings.filterwarnings('ignore', category=UserWarning)

# # Seaborn 테마 설정
# sns.set_theme(style='whitegrid')

# # matplotlib 한글 폰트 설정
# os.system('rm -rf ~/.cache/matplotlib')
# os.system('fc-cache -fv')

# plt.rc('font', family='NanumBarunGothic')
# plt.rcParams['axes.unicode_minus'] = False

In [55]:
# 한글 폰트 사용
font_path = "C:\Windows\Fonts\Hancom Gothic Bold.ttf"  # 경로 확인 후 변경 필요
font_prop = fm.FontProperties(fname=font_path)
plt.rc('font', family='NanumGothic') # For Windows

<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:2: SyntaxWarning: invalid escape sequence '\W'
C:\Users\mir96\AppData\Local\Temp\ipykernel_15260\1615364574.py:2: SyntaxWarning: invalid escape sequence '\W'
  font_path = "C:\Windows\Fonts\Hancom Gothic Bold.ttf"  # 경로 확인 후 변경 필요


# 데이터 수집

In [56]:
df_1 = pd.read_csv("../data/ml_df_join.csv", index_col=0)
df_2 = pd.read_csv("../data/ml_df_rent_fix.csv", index_col=0)
df = pd.merge(df_1, df_2)
x_df = df.dropna()
ml_df_final = x_df.to_csv("../data/ml_df_final.csv") 

In [57]:
# 각 변수에 대한 리스트 작성
store_lst = ['점포수', '유사업종점포수', '개업점포수', '폐업점포수', '프랜차이즈점포수']
pop_floating_lst = ['총유동인구수', '남성유동인구수', '여성유동인구수',
                    '연령대10유동인구수', '연령대20유동인구수', '연령대30유동인구수', '연령대40유동인구수', '연령대50유동인구수', '연령대60이상유동인구수'
                    '시간대0006유동인구수', '시간대0611유동인구수', '시간대1114유동인구수', '시간대1721유동인구수', '시간대2124유동인구수',
                    '월요일유동인구수', '화요일유동인구수', '수요일유동인구수', '목요일유동인구수', '금요일유동인구수', '토요일유동인구수', '일요일유동인구수',
                    ]
pop_stay_lst = ['총상주인구수', '남성상주인구수', '여성상주인구수',
                '연령대10상주인구수', '연령대20상주인구수', '연령대30상주인구수', '연령대40상주인구수', '연령대50상주인구수', '연령대60이상상주인구수'
                '시간대0006상주인구수', '시간대0611상주인구수', '시간대1114상주인구수', '시간대1721상주인구수', '시간대2124상주인구수',
                '월요일상주인구수', '화요일상주인구수', '수요일상주인구수', '목요일상주인구수', '금요일상주인구수', '토요일상주인구수', '일요일상주인구수',
                ]
pop_work_lst = ['총직장인구수', '남성직장인구수', '여성직장인구수',
                '연령대10직장인구수', '연령대20직장인구수', '연령대30직장인구수', '연령대40직장인구수', '연령대50직장인구수', '연령대60이상직장인구수'
                ]
sales_price_lst = ['당월매출금액', '주말매출금액',
                   '월요일매출금액', '화요일매출금액', '수요일매출금액', '목요일매출금액', '금요일매출금액', '토요일매출금액', '일요일매출금액',
                   '시간대0006매출금액', '시간대0611매출금액', '시간대1114매출금액', '시간대1417매출금액', '시간대1721매출금액', '시간대2124매출금액',
                   '남성매출금액', '여성매출금액',
                   '연령대10매출금액', '연령대20매출금액', '연령대30매출금액', '연령대40매출금액', '연령대50매출금액', '연령대60이상매출금액'
                   ]
sales_count_lst = ['당월매출건수', '주말매출건수',
                   '월요일매출건수', '화요일매출건수', '수요일매출건수', '목요일매출건수', '금요일매출건수', '토요일매출건수', '일요일매출건수',
                   '시간대건수00매출건수', '시간대건수06매출건수', '시간대건수11매출건수', '시간대건수14매출건수수', '시간대건수17매출건수', '시간대건수21매출건수',
                   '남성매출건수', '여성매출건수',
                   '연령대10매출건수', '연령대20매출건수', '연령대30매출건수', '연령대40매출건수', '연령대50매출건수', '연령대60이상매출건수'
                   ]
store_stay_lst = ['운영영업개월평균', '폐업영업개월평균']
store_price_lst = ['총계임대료', '1층임대료', '1층외임대료']

## 분기별로 돌리기
- 전체를 돌렸을땐 진짜 90%의 변동을 있게 하지 않는이상 큰변화는 안일어남.
- 분기별로 돌려보고 10%, 20%, 30% 정도 돌려보고 

1번과 2번의 차이점을 알고싶음
- fit을 통해 모델을 저장하고
- 나중에 predict로 예측 진행이 필요

In [58]:
# 데이터 뽑기
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import numpy as np
import pandas as pd

# 데이터 축적용 생성
total_cluster_df = pd.DataFrame(None)

for year in range(2019, 2024):
    for quart in range(1, 5):
        print(f"\n{'='*50}")
        print(f"Processing: {year} Q{quart}")
        print(f"{'='*50}")
        
        # 데이터 전처리
        x_df = x_df.copy()
        
        # 해당 연도와 분기의 컬럼만 선택
        x_df_column = x_df.columns.tolist()
        x_df_mask = [col for col in x_df_column if f'{year}{quart}' in str(col)]
        
        if not x_df_mask:
            print(f"No data found for {year}Q{quart}, skipping...")
            continue
            
        x_train = x_df[x_df_mask].dropna()
        
        if x_train.empty:
            print(f"Empty dataset after dropping NA for {year}Q{quart}, skipping...")
            continue
        
        # 데이터 스케일링
        scaler = StandardScaler()
        data = scaler.fit_transform(x_train)
        
        # 피처 범위 정의
        features = range(data.shape[1])
        
        # 군집분석
        kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
        kmeans_fit = kmeans.fit(data)
        original_labels = kmeans.predict(data)

        # 군집 결과 저장
        cluster_result = x_train.copy()
        cluster_result['cluster'] = original_labels
        
        total_cluster_df[f'{year}{quart}_cluster'] = original_labels


        # 클러스터 1과 2에 해당하는 데이터 인덱스 추출
        cluster_1_2_indices = cluster_result[cluster_result['cluster'].isin([1, 2])].index
        
        # 각 피처별 중요도 저장
        feature_importance = {}
        
        # 각 피처에 대해 테스트
        for feature in features:
            importance_scores = []
            
            # 여러 번의 변화 테스트
            for _ in range(10):  # 반복 횟수 설정
                # 테스트 데이터 복사
                test_data = data.copy()
                
                # 클러스터 1과 2에 해당하는 샘플들만 해당 피처 값 변경
                if len(cluster_1_2_indices) > 0:
                    # 해당 인덱스의 행 번호 찾기 (위치 기반 인덱스로 변환)
                    rows_to_modify = [list(cluster_result.index).index(idx) for idx in cluster_1_2_indices 
                                     if idx in cluster_result.index]
                    
                    # 해당 피처만 약간 변경 (평균의 ±10% 범위 내에서 무작위 변경)
                    feature_std = np.std(data[:, feature])
                    
                    for row in rows_to_modify:
                        # 각 행에 대해 해당 피처만 변경
                        random_change = np.random.normal(0, 0.1 * feature_std)
                        test_data[row, feature] += random_change
                
                # 변경된 데이터로 군집 분석
                test_labels = kmeans.predict(test_data)
                
                # 원래 군집과 새 군집 간의 유사도 측정
                similarity = adjusted_rand_score(original_labels, test_labels)
                importance_scores.append(1 - similarity)  # 유사도가 낮을수록 중요도 높음
            
            # 평균 중요도 계산
            feature_importance[feature] = np.mean(importance_scores)
        
        # 중요도 순으로 정렬
        sorted_importance = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
        
        # 결과 출력
        print(f"피처 중요도 (내림차순): {year}년 {quart}분기 기준")
        for feature, importance in sorted_importance[:20]:  # 상위 20개만 출력
            # 원래 변수명 확인 (컬럼 인덱스를 실제 컬럼명으로 매핑)
            feature_name = x_train.columns[feature]
            print(f"{feature_name}: {importance:.4f}")
        
        # 중요도 분포 확인
        print("\n중요도 통계:")
        importance_values = list(feature_importance.values())
        print(f"최소값: {min(importance_values):.4f}")
        print(f"최대값: {max(importance_values):.4f}")
        print(f"평균값: {np.mean(importance_values):.4f}")
        print(f"중앙값: {np.median(importance_values):.4f}")
        
        # 결과를 DataFrame으로 변환하여 저장
        importance_df = pd.DataFrame(sorted_importance, columns=['Feature_Index', 'Importance'])
        importance_df['Feature_Name'] = importance_df['Feature_Index'].apply(lambda x: x_train.columns[x])
        importance_df = importance_df[['Feature_Index', 'Feature_Name', 'Importance']]
        
        # 결과 저장
        importance_filename = f"feature_importance_{year}_{quart}.csv"
        importance_df.to_csv(f'../data/feature_30_12/{importance_filename}')
        print(f"Saved results to: ../data/feature_30_12/{importance_filename}")

    total_cluster_df.to_csv("../data/total_cluster.csv")


Processing: 2019 Q1
피처 중요도 (내림차순): 2019년 1분기 기준
20191점포수: 0.0000
20191유사업종점포수: 0.0000
20191개업점포수: 0.0000
20191폐업점포수: 0.0000
20191프랜차이즈점포수: 0.0000
20191총유동인구수: 0.0000
20191남성유동인구수: 0.0000
20191여성유동인구수: 0.0000
20191연령대10유동인구수: 0.0000
20191연령대20유동인구수: 0.0000
20191연령대30유동인구수: 0.0000
20191연령대40유동인구수: 0.0000
20191연령대50유동인구수: 0.0000
20191연령대60이상유동인구수: 0.0000
20191시간대0006유동인구수: 0.0000
20191시간대0611유동인구수: 0.0000
20191시간대1114유동인구수: 0.0000
20191시간대1417유동인구수: 0.0000
20191시간대1721유동인구수: 0.0000
20191시간대2124유동인구수: 0.0000

중요도 통계:
최소값: 0.0000
최대값: 0.0000
평균값: 0.0000
중앙값: 0.0000
Saved results to: ../data/feature_30_12/feature_importance_2019_1.csv

Processing: 2019 Q2


KeyboardInterrupt: 

In [ ]:
# # 데이터 

# from sklearn.preprocessing import StandardScaler
# from sklearn.cluster import KMeans
# from sklearn.metrics import adjusted_rand_score
# import numpy as np
# import pandas as pd

# total_cluster_df = pd.DataFrame(None)

# for year in range(2019, 2024):
#     for quart in range(1, 5):
#         print(f"\n{'='*50}")
#         print(f"Processing: {year} Q{quart}")
#         print(f"{'='*50}")
        
#         # 데이터 전처리
#         x_df = pd.read_csv('../data/data_fianl.csv', index_col=0)
        
#         # 해당 연도와 분기의 컬럼만 선택
#         x_df_column = x_df.columns.tolist()
#         x_df_mask = [col for col in x_df_column if f'{year}{quart}' in str(col)]
        
#         if not x_df_mask:
#             print(f"No data found for {year}Q{quart}, skipping...")
#             continue
            
#         x_train = x_df[x_df_mask].dropna()
        
#         # 데이터 스케일링
#         scaler = StandardScaler()
#         data = scaler.fit_transform(x_train)
        
#         # 피처 범위 정의
#         features = range(data.shape[1])
        
#         # 군집분석
#         kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
#         kmeans_fit = kmeans.fit(data)
#         original_labels = kmeans.predict(data)

#         # 군집 결과 저장
#         cluster_result = x_train.copy()
#         cluster_result['cluster'] = original_labels

#         total_cluster_df[f'{year}{quart}_cluster'] = original_labels

# 클러스터 변경

In [ ]:
total_cluster_df['상권'] = x_df['상권'].dropna()
total_cluster_df.head()

,20191_cluster,20192_cluster,20193_cluster,20194_cluster,20201_cluster,20202_cluster,20203_cluster,20204_cluster,20211_cluster,20212_cluster,...,20214_cluster,20221_cluster,20222_cluster,20223_cluster,20224_cluster,20231_cluster,20232_cluster,20233_cluster,20234_cluster,상권
0,3,3,3,0,0,2,0,1,1,1,...,1,2,3,3,0,0,2,1,3,배화여자대학교(박노수미술관)
1,0,1,0,1,1,1,1,0,0,0,...,0,1,0,0,3,2,1,0,0,세종마을음식문화거리(금천교시장)
2,0,1,0,1,1,1,1,0,0,0,...,0,1,0,0,3,2,1,0,0,통인시장
3,0,1,0,1,1,1,1,0,0,0,...,0,1,0,0,3,2,1,0,0,동묘시장(동묘벼룩시장)
4,0,1,0,1,1,1,1,0,0,0,...,0,1,0,0,3,2,1,0,0,동대문문구완구거리(동대문문구완구시장)


In [ ]:
# # 그룹 변경
# change_cluster_df = total_cluster_df.copy()
# join_cluster = pd.DataFrame(None)

# for year in range(2019, 2024):
#     for quart in range(1, 5):
#         change_cluster = change_cluster_df[f'{year}{quart}_cluster'].value_counts().sort_values()
#         change_cluster = change_cluster.reset_index().reset_index()[[f'{year}{quart}_cluster','index']].rename(columns ={'index' : f'{year}{quart}_newcluster'})
#         change_cluster_df = change_cluster_df.merge(change_cluster, left_on = f'{year}{quart}_cluster', right_on = f'{year}{quart}_cluster')
        
# change_df = change_cluster_df.iloc[:,20:]
# column =['상권', '20191_cluster', '20192_cluster', '20193_cluster', '20194_cluster',
#        '20201_cluster', '20202_cluster', '20203_cluster', '20204_cluster',
#        '20211_cluster', '20212_cluster', '20213_cluster', '20214_cluster',
#        '20221_cluster', '20222_cluster', '20223_cluster', '20224_cluster',
#        '20231_cluster', '20232_cluster', '20233_cluster', '20234_cluster']
# change_df.columns = column

# change_df.to_csv = ("../data/New_cluster.csv")

In [ ]:
# 그룹 변경
change_cluster_df = total_cluster_df.copy()

# 결과를 저장할 데이터프레임
result_df = change_cluster_df[['상권']].copy()

# 각 시기별로 클러스터 빈도에 따라 번호 재할당
for year in range(2019, 2024):
    for quart in range(1, 5):
        period = f'{year}{quart}'
        
        # 원래 클러스터 정보 복사
        result_df[f'{period}_cluster_original'] = change_cluster_df[f'{period}_cluster']
        
        # 클러스터 빈도 계산 및 내림차순 정렬 (큰 것부터)
        cluster_counts = change_cluster_df[f'{period}_cluster'].value_counts().sort_values(ascending=True)
        
        # 빈도에 따른 새 번호 매핑 (0부터 시작)
        cluster_mapping = {cluster: i for i, cluster in enumerate(cluster_counts.index)}
        
        # 새 클러스터 번호 할당
        result_df[f'{period}_cluster'] = change_cluster_df[f'{period}_cluster'].map(cluster_mapping)

# 결과 저장
result_df.to_csv("../data/Size_ordered_clusters.csv", index=False)